<a href="https://colab.research.google.com/github/choibyoungjune/pythonbasic/blob/master/%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup as bs
import json
from urllib import parse

#한글 폰트 사용 : colab , 런타임 다시 시작 하기
%config InlineBackend.figure_format = 'retina'

!apt -qq -y install fonts-nanum

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [2]:
def charEncode(st) :
    pt = r'charset[ ]*=[\']*[\"]*([\w-]+)[\"]*[\']*'
    return re.search(pt, st) and re.search(pt, st).group(1) or 'utf-8'
  
def urlHtml(url) :
    url_data = urlopen(url).read()
    urlcc = charEncode(str(url_data))
    html = url_data.decode(urlcc)
    return html

In [3]:
def getMovieInfo(movieId) :
    html_data = urlHtml('https://movie.daum.net/moviedb/grade?movieId='+movieId)
    bs_data = bs(html_data, 'html.parser')

    movie_info = bs_data.select(".list_review > li")

    movie_review = []
    #print(movie_info[4].select(".desc_review")[0].getText())
    for i in movie_info :
        movie_review.append(i.select(".desc_review")[0].getText().lstrip().rstrip())
    
    return movie_review

In [4]:
def getBoxoffice() :
    html_data = urlHtml('https://movie.daum.net/premovie/released')
    bs_data = bs(html_data, 'html.parser')
    
    movie_info = bs_data.select(".list_boxthumb > li")
    pat = r'data-tiara-id=\"([\d]+)\"'

    mvname = []
    for i in movie_info:
        mvname.append(i.select_one('strong').getText())

    dt = {}
    for i in range(len(movie_info)) :
        id = re.search(pat, str(movie_info[i])).group(1)
        dt[mvname[i]] = getMovieInfo(id)
        #print(mvname[i]," ==> ", getMovieInfo(id)[0])
    
    return dt

In [5]:
dt = getBoxoffice()

#print(dt["#살아있다"])
with open('/content/drive/My Drive/Colab/해결문제/movie_review.txt', 'w', encoding='utf-8') as fp:
    for i in dt.keys():
        content = str(i)
        for j in dt[i]:
            content = str(i) + ', ' + j + '\n'
            fp.write(content)

In [6]:
def callAdamsOM(myquery, mytype) :
  mykey = '7927500106886487277'

  url = 'http://api.adams.ai/datamixiApi/omAnalysis?key=' + mykey
  url = url + '&query=' + parse.quote(myquery)
  url = url + '&type=' + str(mytype)
  data = urlopen(url).read().decode('utf-8')
  dt = json.loads(data)
  return dt['return_object']['label'], dt['return_object']['score']

In [8]:
from collections import defaultdict
dt=defaultdict(list)

rt={'긍정':0, '부정':0, '중립':0}
with open('/content/drive/My Drive/Colab/해결문제/movie_review.txt', 'r', encoding='utf-8') as fp:
    lst = fp.readlines()

print(len(lst), lst)

for i in lst :
    if len(i)>140 :
        i=i[:140]
    print(len(i))

    label, score = callAdamsOM(i, 0)
    i=i.replace('\n','')
    print(f"'{i}' 의 감정은 ==> {label} : {score : .2f} 입니다.")
    rt[label] = rt[label]+1
    dt[label].append(i)

print(f"긍정 : {len(dt['긍정'])}, 중립 : {len(dt['중립'])}, 부정 : {len(dt['부정'])}")

263 ['#살아있다, 좀비 영화 다 비슷한 내용이지~ 너무 기대들 하시고 보신것 같음\n', '나름 좀비 드라마인듯 가벼운 맘으로 보면 볼만하다 생각했음\n', '깜짝놀랄만한게 나오나했는데 역시나 그 장면은 심장 쫄깃했음\n', '#살아있다, 코로나 시기에 일부러 시간내서 돈 내서 보러 갔는데 무슨 tv드라마보다 재미가 없노.\n', '#살아있다, 평점이 5점이나 되나요? 너무 높아서 원래 2점짜리인데, 0점 줍니다.\n', '#살아있다, 재미없음\n', '#살아있다, 내용은 없으나 연기력은 짱\n', '#살아있다, 우리 살아있다\n', '#살아있다, 볼 영화가 없어서 예매율 1위인 영화.\n', '예고편이 전부인 영화임.\n', '좀비가 한국에서 뜨니까 만들어진 아류작이며 보는 내내 번히 예성되는 내용, 뻔한  전개 때문에 극장 빈자리를 둘러보면서 하품하기는 처음.\n', '고등학교, 대학교 영화동아리에서 으싸 으싸햐서 여름 방학동안 만든 영화 같음.\n', '영화 음악도 어디서 둘운거 같은데??? 생각이 듬\n', '#살아있다, \n', '#살아있다, \n', '#살아있다, 내용은 평범한데 유아인 연기력의 점수준거임\n', '밤쉘: 세상을 바꾼 폭탄선언, <빅 쇼트> 제작진의 또 하나의 사회를 향해 던지는 돌직구.\n', '밤쉘: 세상을 바꾼 폭탄선언, 실화가 바탕인 때문인지 이야기의 나열이 주를 이루고, 제대로 된 처벌 없이 퇴사와 합의로 끝을 맺기에 통쾌함이 덜하다. 그리고 메긴 켈리를 비롯한 과거 피해자들의 증언과 그레천의 녹음 파일만으로는 로저 에일스의 악행을 표현하기에 부족하다고 여겼는지 케일라(마고 로비)라는 가상의 인물을 넣었는데, 오히려 이야기의 신빙성을 떨어뜨릴 수도 있어 과유불급이라는 생각이 든다. 성별을 떠나 권력을 남용해 사심을 채우는 일은 결코 없어야 한다. 용기 있는 폭로가 성 문제뿐 아니라 모든 분야에서 일어나길 바란다. PS:문득 폭로(1994)가 떠올랐다.\n', '밤쉘: 세상을 바꾼 폭탄선언, \n', '밤쉘: 세상을 바꾼

KeyError: ignored